# Sanity check for mnist

Run some mnist models through the test set and visualize results

In [4]:
import os
import yaml
from hparam_tuning_project.training.utils import build_modules
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
matplotlib.use('Qt5Agg')
experiment = "test_lr_results_on_full_datasets"
version = "version_1"
batch_size = 24
checkpoint_root = f"./experiments/{experiment}/lightning_logs/{version}/"
checkpoint_path = os.path.join(checkpoint_root, "checkpoints", os.listdir(checkpoint_root + "checkpoints")[0])
with open(os.path.join(checkpoint_root, "hparams.yaml"), "r") as f:
    cfg = yaml.safe_load(f)

cfg['data_cfg']['batch_size'] = batch_size
learner, model, dataset = build_modules(cfg, checkpoint_path=checkpoint_path)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


pl version =  2.4.0
model is LightningModule? True


In [5]:
def get_inferences(results):
    results = F.sigmoid(results)
    indices = torch.argmax(results, dim=1)
    return indices

dataset.setup()
dl = dataset.test_dataloader()
sample, label = None, None
for _sample, _label in dl:
    sample = _sample
    label = _label
    break
results = model(sample)
results = get_inferences(results)

In [6]:
nrows = batch_size // 4
ncols = 4
fig, axes = plt.subplots(nrows=nrows, ncols=ncols)
for i, ax in enumerate(axes.flatten()):
    img = sample[i][0, :, :]
    pred = results[i]
    gt = label[i]
    ax.imshow(img, cmap='gray')
    if pred == gt:
        color = 'green'
    else:
        color = 'red'
    ax.set_title(f"pred = {pred}, actual = {gt}", color=color)
plt.show()